## PROGRAM extrct
Extracts data from the N-body output files Single.dat (fort.83) and Binary.dat (fort.82) relating to the overall cluster evolution, ie. half-mass radius and relaxation time, cluster mass, membership, core radius. 

Note: planets are ignored in this version. 

Output in extrct.dat: 

*   Number of single stars + binaries 
*   Number of binaries 
*   Time (Myr) 
*   Relaxation time (Myr) 
*   Total cluster mass (Msun) 
*   Mass in core (Msun) 
*   Mass outside the tidal radius (Msun) 
*   Maximum stellar distance from cluster centre of mass (pc) 
*   Half-mass radius (pc) 
*   Radius containing inner 10% of cluster mass (pc) 
*   Core radius - as determined by Nbody code (pc) 
*   Number of systems (stars + binaries) inside the half-mass radius   
*   Number of systems within 1pc of the cluster centre    
*   Number of systems within the inner lagrangian radius (10%)  
*   Number of systems within the core radius 
*   Velocity dispersion (km/s)

In [43]:
#Necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

## Reformatting and reading data from fort.83 and fort.82
(Removing format of fort.83 and fort.82 - run this just once)

In [2]:
import os.path

file83 = os.path.isfile('fort.83-awk')
file82 = os.path.isfile('fort.82-awk')

if not file83:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' fort.83 > fort.83-awk
    print "Removing format of fort.83"
if not file82:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22,$23,$24}' fort.82 > fort.82-awk
    print "Removing format of fort.82"

In [44]:
Ns = 0
Nb = 0
with open("fort.83-awk", 'r') as file1:
    with open("fort.82-awk", 'r') as file2:
        while file1 and file2:
            line11 = file1.readline()
            line21 = file2.readline()
            if not (line11 and line21):
                break
            words11 = line11.split()
            words21 = line21.split()
            Ns = int(words11[0])
            Nb = int(words21[0])
            if Ns != -1000:
                line12 = file1.readline()
                words12 = line12.split()
                line13 = file1.readline()
                words13 = line13.split()
                blockS = np.fromfile(file1, sep=' ', count=13*Ns).reshape([Ns,13])
            if Nb != -1000:
                blockB = np.fromfile(file2, sep=' ', count=24*Nb).reshape([Nb,24])

In [50]:
#Numerical initiation of constants
amin = 1.0e+10
amax = -1.0e+10
vstar = -1
trhsum = 0
iprsum = -1
m0 = 0
rmax = 1
mtot = 0
mout = 0
n1 = 0
n2 = 0
n1pc = 0
vdisp = 0
msgl = 0
nsgl = 0
mwd = 0.0
nwd = 0


In [57]:
#Astronomical constants
pc = 3.0856776e+18
au = pc/1.4959787e+12

In [ ]:
#Main call of functions and writing to output file

In [56]:
def globaldata():
    m0 = float(words13[0])
    rbar = words12[3]
    zmbar = words13[1]
    su = pc/(au*6.955e+10)*rbar*au
    if vstar == 0:
        vstar = 0.06557d0*SQRT(m0/rbar)
        print ' ZMBAR RBAR VSTAR ',m0,rbar,vstar
        ebcut = (1./zmbar)*(1./zmbar)/(20/su)

3.0856776e+18


In [13]:
#OPEN(3,file='extrct.dat',status='unknown')

False